In [6]:
#Sample snowpark use case that connects to an api to do simple transformations on a snowflake sample table 
#and load into temp_db database

#Importing required libraries
from snowflake.snowpark import Session
import requests
import pandas as pd
import os
from dotenv import load_dotenv

In [7]:
#load environment variables
load_dotenv()

api_key = os.getenv("api_key")
account = os.getenv("account")
user = os.getenv("user")
password = os.getenv("password")
role = os.getenv("role")
warehouse = os.getenv("warehouse")
database = os.getenv("database")
schema = os.getenv("schema")

In [8]:
#Connecting to snowflake
connection_parameters = {
"account": account,
"user": user,
"password": password,
"role": role,
"warehouse": warehouse,
"database": database,
"schema": schema
}

#Creating a snowflake session
session = Session.builder.configs(connection_parameters).create()

In [9]:
#Creating a snowflake dataframe
df_table = session.table("SNOWFLAKE_SAMPLE_DATA.TPCDS_SF10TCL.WAREHOUSE")
df_table.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"W_WAREHOUSE_SK"  |"W_WAREHOUSE_ID"  |"W_WAREHOUSE_NAME"    |"W_WAREHOUSE_SQ_FT"  |"W_STREET_NUMBER"  |"W_STREET_NAME"  |"W_STREET_TYPE"  |"W_SUITE_NUMBER"  |"W_CITY"       |"W_COUNTY"         |"W_STATE"  |"W_ZIP"  |"W_COUNTRY"    |"W_GMT_OFFSET"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1                 |AAAAAAAABAAAAAAA  |Conventional childr   |977787               |651                |6th              |Parkway          |Suite 470         |Shiloh         |Franklin Parish    |LA         |79275    |United States  |-6.00         

In [11]:
#Converting it to pandas df
df_pd = df_table.to_pandas()
df_pd.head()

,W_WAREHOUSE_SK,W_WAREHOUSE_ID,W_WAREHOUSE_NAME,W_WAREHOUSE_SQ_FT,W_STREET_NUMBER,W_STREET_NAME,W_STREET_TYPE,W_SUITE_NUMBER,W_CITY,W_COUNTY,W_STATE,W_ZIP,W_COUNTRY,W_GMT_OFFSET
0,1,AAAAAAAABAAAAAAA,Conventional childr,977787.0,651,6th,Parkway,Suite 470,Shiloh,Franklin Parish,LA,79275,United States,-6.0
1,2,AAAAAAAACAAAAAAA,Important issues liv,138504.0,600,View First,Avenue,Suite P,Greenwood,Ziebach County,SD,58828,United States,-6.0
2,3,AAAAAAAADAAAAAAA,Doors canno,294242.0,534,Ash Laurel,Dr.,Suite 0,Cedar Grove,Raleigh County,WV,20411,United States,-5.0
3,4,AAAAAAAAEAAAAAAA,Bad cards must make.,621234.0,368,Wilson Elm,Drive,Suite 80,Oakland,Gage County,NE,69843,United States,-6.0
4,5,AAAAAAAAFAAAAAAA,None,NaN,None,None,None,None,Midway,Mesa County,CO,81904,United States,NaN


In [12]:
#Getting weather data from API for every zipcode
json_list = []
try:
    for i in df_pd['W_ZIP']:
        response = requests.get(f"http://api.openweathermap.org/geo/1.0/zip?zip={i},US&appid={api_key}")
        data = response.json()
        json_list.append(data)
except ValueError:
    pass

In [13]:
#Creating a new df with weather info and zip code
zip_code = df_pd['W_ZIP']
data = list(zip(zip_code, json_list))
weather_df = pd.DataFrame(data=data, columns=['W_ZIP', 'WEATHER_JSON'])

In [14]:
#Joining weather nad warehouse tables
snowflake_df = df_pd.merge(weather_df, how='inner', on='W_ZIP')

In [15]:
#Writing table to snowflake
snowflake_df_1 = session.create_dataframe(snowflake_df)
snowflake_df_1.write.mode("overwrite").save_as_table("TEMP_DB.PUBLIC.warehouse_weather_api")

create_temp_table is deprecated. We still respect this parameter when it is True but please consider using `table_type="temporary"` instead.


In [ ]:
#End of Code